In [13]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.callbacks import ModelCheckpoint

In [14]:
class SmallerVGGNet:
    @staticmethod #change activation to sigmoid for multi class multi label classification
    def build(width, height, depth, classes, finalAct='softmax'): #depth = num of channels
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32 , (3, 3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64 , (3, 3), padding="same")) #, input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64 , (3, 3), padding="same")) #, input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128 , (3, 3), padding="same")) #, input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same")) #, input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
#the change in filters, kernels, poolsizes progressively reduce spatial size but increase depth!
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(classes))
        model.add(Activation(finalAct))
        
        return model
    

In [15]:
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
#import pickle #not done this time
import cv2
import os
from tqdm import tqdm

In [16]:
epochs = 20
lr = 0.001
batch_size = 32
image_size = (96, 96, 3)

In [17]:
imagePaths = sorted(list(paths.list_images("dataset")))
random.seed(42)
random.shuffle(imagePaths)

data = []
labels = []


In [18]:
print("#loading images...")

unique = []
rest= []

for imagepath in tqdm(imagePaths):
    img = cv2.imread(imagepath)
    img = cv2.resize(img, (image_size[1], image_size[0]))
    img = img_to_array(img)
    data.append(img)
    l = imagepath.split(os.path.sep)[-2].split("_")
    other = imagepath.split(os.path.sep)[-2]
    if other not in rest:
        rest.append(other)
    if l not in unique:
        unique.append(l)
    labels.append(l)

  0%|          | 6/2165 [00:00<00:53, 40.69it/s]

#loading images...


100%|██████████| 2165/2165 [01:13<00:00, 29.65it/s]


In [19]:
print(rest)

['blue_dress', 'red_shirt', 'blue_jeans', 'red_dress', 'blue_shirt', 'black_jeans']


In [9]:
print(unique,labels,sep='\n\n')

[['blue', 'dress'], ['red', 'shirt'], ['blue', 'jeans'], ['red', 'dress'], ['blue', 'shirt'], ['black', 'jeans']]

[['blue', 'dress'], ['red', 'shirt'], ['blue', 'jeans'], ['blue', 'dress'], ['red', 'dress'], ['red', 'dress'], ['blue', 'shirt'], ['black', 'jeans'], ['blue', 'shirt'], ['red', 'shirt'], ['blue', 'jeans'], ['red', 'dress'], ['red', 'shirt'], ['blue', 'dress'], ['black', 'jeans'], ['red', 'dress'], ['red', 'dress'], ['blue', 'shirt'], ['blue', 'jeans'], ['red', 'shirt'], ['blue', 'dress'], ['red', 'dress'], ['red', 'dress'], ['blue', 'shirt'], ['blue', 'shirt'], ['red', 'dress'], ['blue', 'jeans'], ['black', 'jeans'], ['blue', 'jeans'], ['red', 'shirt'], ['red', 'dress'], ['blue', 'jeans'], ['red', 'shirt'], ['blue', 'shirt'], ['blue', 'jeans'], ['blue', 'shirt'], ['red', 'shirt'], ['red', 'shirt'], ['red', 'shirt'], ['blue', 'dress'], ['blue', 'jeans'], ['blue', 'dress'], ['red', 'shirt'], ['blue', 'shirt'], ['blue', 'jeans'], ['black', 'jeans'], ['red', 'dress'], ['blue'

In [10]:
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

print('# Data matrix: {} images ({:.2f}MB)'.format(len(imagePaths), data.nbytes / (1024*1000.0)))

# Data matrix: 2165 images (467.64MB)


In [20]:
print('#Class labels : ')
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

for (i, label) in enumerate(mlb.classes_):
    print('{}. {}'.format(i+1, label))

#Class labels : 
1. black
2. blue
3. dress
4. jeans
5. red
6. shirt


In [28]:
print(type(mlb.classes_[0]))

<class 'str'>


In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict, average='weighted')
        _val_recall = recall_score(val_targ, val_predict, average='weighted')
        _val_precision = precision_score(val_targ, val_predict, average='weighted')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("— val_f1: {} — val_precision: {} — val_recall {}".format(_val_f1, _val_precision, _val_recall))
        return
metrics = Metrics()
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data
    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(data, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test1, y_test1, test_size=0.2, random_state=42)
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [ ]:
model = SmallerVGGNet.build(width=image_size[1], height = image_size[0], depth=image_size[2], classes=len(mlb.classes_), finalAct="sigmoid")
opt = Adam(lr=lr, decay = lr/epochs)

The kernel had a crashing problems on this jupyter notebook at MacOS, but works perfect else where..

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

checkpoint = ModelCheckpoint('{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

#since the data is already has almost equal distribution, class imbalance handling measurements are not required

print('#Training Network..')
H = model.fit_generator(aug.flow(X_train, y_train, batch_size=batch_size),
                       validation_data = (X_valid, y_valid), steps_per_epoch = len(X_train)//batch_size, epochs=50, verbose=1, callbacks=[metrics, TestCallback(X_valid, y_valid)])


In [ ]:
plt.style.use('ggplot')
plt.figure()
n = epochs
plt.plot(np.arange(0, n), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, n), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, n), H.history['acc'], label='train_acc')
plt.plot(np.arange(0, n), H.history['val_acc'], label='val_acc')
plt.title("Training and Validation : Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss_and_Accuracy")
plt.legend(loc="upper left")
plt.savefig('Acc_loss_graph.png')

# Prediction

In [ ]:
from keras.models import load_model

#if required
# model = load_model('0.9942.hdf5')
# mlbclasses_ = np.array(['black','blue','dress','jeans','red','shirt'])

mlbclasses_ = ['black','blue','dress','jeans','red','shirt']

image = cv2.imread('model.jpg')
output = imutils.resize(image, width=400)

image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
print(image.shape)
image = np.expand_dims(image, axis=0)
print('Expanded dims:', image.shape)

In [ ]:
proba = model.predict(image)[0]
# idxs = np.argsort(proba)[::-1][:2] #extract top 2 labels, has problem at macOS jupyter notebook

for (i, j) in enumerate(idxs):
    label = "{}: {:.2f}%".format(mlbclasses_[j], proba[j]*100)
    cv2.putText(output, label, (10, (i*30)+25 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

for (label, p) in zip(mlbclasses_, proba):
    print("{}: {:2f}%".format(label, p*100))
    
# cv2.imshow("Output", output) #a few problems at MacOS jupyter notebook
# cv2.waitKey(0) 